In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

df = pd.read_pickle("../data/engage_rating.pkl")
df

,user_id,item_id,rating,timestamp,parent_asin
0,0,7124,5.0,1143213544000,B0007MGG2M
0,0,9902,2.0,1174661490000,B000ELSZBQ
0,0,12453,5.0,1275251437000,B000JREMTO
0,0,16792,5.0,1530219104429,B000WYVBR0
0,0,17034,5.0,1234625406000,B000XT3L7W
...,...,...,...,...,...
1641025,1641025,165869,1.0,1600221333878,B01H6VDNRW
1641025,1641025,206156,5.0,1600220478096,B076P16449
1641025,1641025,250317,5.0,1584750059535,B07QQ4YB76
1641025,1641025,331214,5.0,1600221792087,B09W4W9SYF


In [8]:
df["user_id"].max(), df["item_id"].max()

(1641025, 368227)

In [9]:
user_encode_df = df.groupby("user_id").agg({
    "item_id": lambda x: x.tolist(),
    "rating": lambda x: x.tolist()
}).reset_index(drop=False)

user_encode_df

,user_id,item_id,rating
0,0,"[7124, 9902, 12453, 16792, 17034, 25524, 27164...","[5.0, 2.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, ..."
1,1,"[252503, 289438, 346023, 359490, 361820]","[4.0, 4.0, 5.0, 5.0, 4.0]"
2,2,"[28620, 32476, 42885, 44496, 50977, 58947, 856...","[4.0, 4.0, 4.0, 4.0, 5.0, 4.0, 4.0, 5.0, 4.0, ..."
3,3,"[56322, 59685, 63968, 77946, 81434, 89536]","[4.0, 4.0, 5.0, 5.0, 4.0, 5.0]"
4,4,"[123339, 153649, 170557, 237478, 352971]","[5.0, 5.0, 5.0, 5.0, 5.0]"
...,...,...,...
1641021,1641021,"[83667, 89640, 91093, 103978, 112033, 117034, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 2.0, 5.0, 5.0, 5.0, ..."
1641022,1641022,"[275954, 277121, 299110, 361589, 364626]","[5.0, 4.0, 5.0, 5.0, 5.0]"
1641023,1641023,"[20577, 21941, 29486, 30900, 117974, 119654, 1...","[5.0, 5.0, 2.0, 5.0, 5.0, 5.0, 5.0, 2.0, 5.0, ..."
1641024,1641024,"[90193, 90700, 105456, 112630, 212251, 273075,...","[5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 5.0]"


In [11]:
user_encode_df["item_id"][0]

[7124,
 9902,
 12453,
 16792,
 17034,
 25524,
 27164,
 35517,
 39994,
 47464,
 52486,
 56392,
 68453,
 72252,
 76050,
 82585,
 110824,
 112571,
 181986,
 301610,
 342412]

In [12]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

item_sequences = rating_df['item_id'].tolist()  # item_id를 리스트로 변환
user_ids = rating_df['user_id'].tolist()


# 데이터셋 클래스 정의
class SessionDataset(Dataset):
    def __init__(self, sequences):
        self.sequences = sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        return torch.tensor(sequence[:-1]), torch.tensor(sequence[-1])
    
# 데이터 분리 (train: 80%, test: 20%)
train_data, test_data = train_test_split(item_sequences, test_size=0.2, random_state=42)

# Hyperparameters
embedding_dim = 128
hidden_dim = 128
num_epochs = 1000
learning_rate = 1e-4
batch_size = 512

train_loader = DataLoader(SessionDataset(train_data), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(SessionDataset(test_data), batch_size=batch_size, shuffle=False)

NameError: name 'rating_df' is not defined

In [ ]:
import torch.optim as optim
class GRURecommender(nn.Module):
    def __init__(self, num_items, embedding_dim, hidden_dim):
        super(GRURecommender, self).__init__()
        self.embedding = nn.Embedding(num_items, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_items)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.gru(x)
        x = self.relu(x[:, -1, :])
        x = self.fc(x)
        return x

num_items = 368228

# 모델 인스턴스화
model = GRURecommender(num_items, embedding_dim, hidden_dim).cuda()

# Loss와 Optimizer 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
from sklearn.metrics import precision_score, recall_score
from tqdm import tqdm
from copy import deepcopy
import warnings
warnings.filterwarnings('always')

model_parameters = deepcopy(model.state_dict())

def precision_at_k(preds, target, k=20):
    # Get top k predictions
    top_k_preds = preds.topk(k, dim=1).indices.cpu().numpy()
    target = target.cpu().numpy()
    
    # Vectorized calculation of precision
    y_true = np.isin(top_k_preds, target[:, None])  # Check if target is in the top_k_preds
    y_pred = np.ones_like(y_true)  # All predictions are considered relevant
    
    precision_scores = []
    for true, pred in zip(y_true, y_pred):
        if np.sum(pred) == 0:  # Avoid division by zero
            precision_scores.append(0)
        else:
            precision_scores.append(precision_score(true, pred, zero_division=0))
    return np.mean(precision_scores)

def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, num_epochs, early_stopping_patience=3):
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, target in tqdm(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs.cuda())
            loss = criterion(outputs, target.cuda())

            loss.backward()
            optimizer.step()
            total_loss += loss.detach().cpu().item()

        val_loss, val_precision = evaluate(model, val_loader, criterion)
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}, '
              f'Val Loss: {val_loss:.4f}, Precision@20: {val_precision:.4f}')
        
        # Early stopping
        if val_loss < best_val_loss:
            print("?")
            best_val_loss = val_loss
            patience_counter = 0
            model_parameters = deepcopy(model.state_dict())
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print("Early stopping triggered")
                break

def evaluate(model, data_loader, criterion):
    model.eval()
    total_loss = 0
    total_precision = 0
    total_samples = 0
    
    with torch.no_grad():
        for inputs, target in tqdm(data_loader):
            outputs = model(inputs.cuda())
            loss = criterion(outputs, target.cuda())
            total_loss += loss.detach().cpu().item()
            
            # Precision@20 and Recall@20
            total_precision += precision_at_k(outputs, target, k=20) * inputs.size(0)
            total_samples += inputs.size(0)
            
    avg_loss = total_loss / len(data_loader)
    avg_precision = total_precision / total_samples
    return avg_loss, avg_precision

# 모델 학습 및 평가
train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, num_epochs)


 43%|████▎     | 27/63 [00:19<00:26,  1.36it/s]

In [ ]:
model.load_state_dict(model_parameters)

avg_loss, avg_precision = evaluate(model, test_loader, criterion)
print(avg_loss, avg_precision)

100%|██████████| 63/63 [00:43<00:00,  1.44it/s]

12.824420323447576 0.0013712416656183166


In [ ]:
print(f'Val Loss: {avg_loss:.4f}, Precision@20: {avg_precision:.4f}')

Val Loss: 12.8244, Precision@20: 0.0014
